In [1]:
import numpy as np
import pandas as pd
import config_final
import requests
from bs4 import BeautifulSoup as BS


In [2]:
config_final.user

'munzma'

# Webscrape URL List for Bill Summaries

In [3]:
raw = pd.read_csv('bills93-114.csv', sep=";", encoding='latin-1')
raw.dropna(axis=0, subset = ['Cosponsr'], inplace=True)

#raw['URL'] = raw['URL'].astype('str') 
# urls = df[['BillID','URL']]
# df = df.set_index('BillID')

print(raw.shape)

(294485, 48)


In [4]:
len(raw)

294485

## Create Scraper

In [5]:
from bs4 import NavigableString

def scraper(url):
       
    response = requests.get(url)
    soup = BS(response.content, "html.parser")
    soup.prettify() 
    
    bill_summary = soup.find('div', id="bill-summary")
    paragraphs = bill_summary.find_all('p')
    
    if len(paragraphs) > 0 and paragraphs[0].get_text(strip=True).startswith('There is'):
        paragraphs = paragraphs[1:]

    if len(paragraphs) > 0:
        return paragraphs[0].get_text(' ', strip=True)
    else:
        node = bill_summary.contents[-1]
        if type(node) == NavigableString:
            return node.strip()
        else:
            print(url) 
            return ''



In [87]:
def clean_df(df):   
    
    df['Class'].fillna(0, inplace=True)

    df['PooleID'].fillna(0, inplace=True)

    df['PLawDate'].fillna(0, inplace=True)

    df['PLawNum'].fillna(0, inplace=True)

    df['Major'].fillna(0, inplace=True)

    df['Minor'].fillna(0, inplace=True)
    
    df['URL'].fillna('', inplace=True)
    
    df = df[df['BillType'].isin(['hr', 's']) ]
    return df

In [88]:
clean_df(raw).head()

,BillID,BillNum,BillType,Chamber,Cong,Cosponsr,IntrDate,Mult,MultNo,PLaw,...,PLawDate,PLawNum,ImpBill,SubChRef,SubRankRef,Majority,ReportH,ReportS,Major,Minor
1880,93-HR-1,1,hr,0,93,0.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,3.0,301.0
1881,93-HR-2,2,hr,0,93,1.0,1973-01-03,0.0,1.0,1,...,1974-09-02,Public law 93-406.,1.0,0.0,0.0,1,1,0,5.0,503.0
1882,93-HR-3,3,hr,0,93,2.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,8.0,805.0
1883,93-HR-4,4,hr,0,93,21.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,2.0,208.0
1884,93-HR-5,5,hr,0,93,0.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,6.0,601.0


In [89]:
df = clean_df(raw)

In [90]:
df['BillType'].value_counts()

hr    164698
s      72919
Name: BillType, dtype: int64

In [91]:
df.shape

(237617, 48)

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237617 entries, 1880 to 293782
Data columns (total 48 columns):
BillID        237617 non-null object
BillNum       237617 non-null int64
BillType      237617 non-null object
Chamber       237617 non-null int64
Cong          237617 non-null int64
Cosponsr      237617 non-null float64
IntrDate      237617 non-null object
Mult          237617 non-null float64
MultNo        237617 non-null float64
PLaw          237617 non-null int64
Private       237617 non-null int64
Title         237617 non-null object
Veto          237617 non-null int64
Class         237617 non-null float64
ComC          237617 non-null float64
ComR          237617 non-null float64
CumHServ      237617 non-null float64
CumSServ      237617 non-null float64
Delegate      237617 non-null int64
District      237617 non-null float64
DW1           237431 non-null object
FrstConH      237617 non-null float64
FrstConS      237617 non-null float64
Gender        237617 non-null i

## Enter into Database

In [93]:
from sodapy import Socrata
import sqlalchemy as db

In [94]:
def query(q):
    try:
        return pd.read_sql_query(q, engine)
    # Pass when no data is returned    
    except ResourceClosedError:
        pass

def query_list(col, table, distinct = True):
    elts = ['SELECT',
            'DISTINCT' if distinct else '',
            col,
            'FROM',
            table]
    query_str = ' '.join(elts)
    df = query(query_str)
    l = df.iloc[:,0].tolist()
    return l

**Create**

In [95]:
#instantiate the connection and table

engine = db.create_engine(f'mysql+mysqlconnector://{config_final.user}:{config_final.password}@{config_final.host}/con_bills')

connection = engine.connect()
metadata=db.MetaData()

Exception during reset or similar
Traceback (most recent call last):
  File "/Users/melissamunz/anaconda3/lib/python3.7/site-packages/sqlalchemy/pool/base.py", line 680, in _finalize_fairy
    fairy._reset(pool)
  File "/Users/melissamunz/anaconda3/lib/python3.7/site-packages/sqlalchemy/pool/base.py", line 867, in _reset
    pool._dialect.do_rollback(self)
  File "/Users/melissamunz/anaconda3/lib/python3.7/site-packages/sqlalchemy/dialects/mysql/base.py", line 2241, in do_rollback
    dbapi_connection.rollback()
  File "/Users/melissamunz/anaconda3/lib/python3.7/site-packages/mysql/connector/connection_cext.py", line 369, in rollback
    self._cmysql.rollback()
_mysql_connector.MySQLInterfaceError: Lost connection to MySQL server during query


In [96]:
#Create Table 
current_bills_table = db.Table('current_bills', metadata,               
    db.Column('BillID', db.String(40), primary_key=True),
    db.Column('BillType', db.String(20)),
    db.Column('Chamber', db.Boolean()),
    db.Column('Cong', db.Integer()),
    db.Column('Cosponsr', db.Float()),
    db.Column('IntrDate', db.DateTime()),
    db.Column('Mult', db.Boolean()),
    db.Column('PLaw', db.Boolean()),
    db.Column('Title', db.String(500)),
    db.Column('Veto', db.Boolean()),
    db.Column('Class', db.Float()),
    db.Column('District', db.Float()),
    db.Column('FrstConH', db.Float()),
    db.Column('FrstConS', db.Float()),
    db.Column('Gender', db.Boolean()),
    db.Column('MRef', db.Boolean()),
    db.Column('NameFull', db.String(40)),
    db.Column('Party', db.Float()),
    db.Column('Postal', db.String(10)),
    db.Column('PassH', db.Boolean()),
    db.Column('PassS', db.Boolean()),
    db.Column('PLawDate', db.DateTime()),
    db.Column('PLawNum', db.String(40)),
    db.Column('ImpBill', db.Boolean()),
    db.Column('Majority', db.Boolean()),
    db.Column('Major', db.Float()),
    db.Column('Minor', db.Float()),
    db.Column('URL', db.String(100)),
    db.Column('Summary', db.String(2000))
                         
)

In [97]:
metadata.create_all(engine)

In [98]:
df.head()

,BillID,BillNum,BillType,Chamber,Cong,Cosponsr,IntrDate,Mult,MultNo,PLaw,...,PLawDate,PLawNum,ImpBill,SubChRef,SubRankRef,Majority,ReportH,ReportS,Major,Minor
1880,93-HR-1,1,hr,0,93,0.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,3.0,301.0
1881,93-HR-2,2,hr,0,93,1.0,1973-01-03,0.0,1.0,1,...,1974-09-02,Public law 93-406.,1.0,0.0,0.0,1,1,0,5.0,503.0
1882,93-HR-3,3,hr,0,93,2.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,8.0,805.0
1883,93-HR-4,4,hr,0,93,21.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,2.0,208.0
1884,93-HR-5,5,hr,0,93,0.0,1973-01-03,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,6.0,601.0


In [99]:
df = df.reindex(index=df.index[::-1])

In [100]:
df.shape

(237617, 48)

In [101]:
df['URL'].isnull().sum()

0

In [102]:
df.head()

,BillID,BillNum,BillType,Chamber,Cong,Cosponsr,IntrDate,Mult,MultNo,PLaw,...,PLawDate,PLawNum,ImpBill,SubChRef,SubRankRef,Majority,ReportH,ReportS,Major,Minor
293782,114-S-3548,3548,s,1,114,3.0,2016-12-10,0.0,1.0,0,...,0,0,1.0,0.0,0.0,0,0,0,3.0,333.0
293781,114-S-3547,3547,s,1,114,0.0,2016-12-09,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,20.0,2002.0
293780,114-S-3546,3546,s,1,114,0.0,2016-12-09,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,9.0,900.0
293779,114-S-3545,3545,s,1,114,0.0,2016-12-09,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,12.0,1209.0
293778,114-S-3544,3544,s,1,114,0.0,2016-12-09,0.0,1.0,0,...,0,0,1.0,0.0,0.0,1,0,0,12.0,1209.0


In [103]:
#Test - Convert to dictionary format
final = df.to_dict(orient = 'records')
final[0]

{'BillID': '114-S-3548',
 'BillNum': 3548,
 'BillType': 's',
 'Chamber': 1,
 'Cong': 114,
 'Cosponsr': 3.0,
 'IntrDate': '2016-12-10',
 'Mult': 0.0,
 'MultNo': 1.0,
 'PLaw': 0,
 'Private': 0,
 'Title': 'A bill to continue the Medicaid emergency psychiatric demonstration project.',
 'Veto': 0,
 'Class': 1.0,
 'ComC': 0.0,
 'ComR': 1.0,
 'CumHServ': 7277.0,
 'CumSServ': 4382.0,
 'Delegate': 0,
 'District': 0.0,
 'DW1': '-0,33',
 'FrstConH': 0.0,
 'FrstConS': 0.0,
 'Gender': 0,
 'MemberID': '15408-114-100',
 'MRef': 0,
 'NameFirst': 'Benjamin',
 'NameFull': 'Benjamin Cardin',
 'NameLast': 'Cardin',
 'Party': 100.0,
 'PooleID': 15408.0,
 'Postal': 'MD',
 'State': 52,
 'URL': 'https://www.congress.gov/bill/114th-congress/senate-bill/3548',
 'ChRef': 0,
 'RankRef': 0,
 'PassH': 0,
 'PassS': 0,
 'PLawDate': 0,
 'PLawNum': 0,
 'ImpBill': 1.0,
 'SubChRef': 0.0,
 'SubRankRef': 0.0,
 'Majority': 0,
 'ReportH': 0,
 'ReportS': 0,
 'Major': 3.0,
 'Minor': 333.0}

In [116]:
completed = 22634
errored = 3
next_index = completed + errored
remaining_final = final[next_index:-1]

{'BillID': '112-HR-3038',
 'BillNum': 3038,
 'BillType': 'hr',
 'Chamber': 0,
 'Cong': 112,
 'Cosponsr': 0.0,
 'IntrDate': '9/23/2011',
 'Mult': 0.0,
 'MultNo': 0.0,
 'PLaw': 0,
 'Private': 0,
 'Title': 'To amend title 49, United States Code, to direct the Secretary of Homeland Security to approve applications submitted by airport operators for participation in the security screening opt-out program, and for other purposes.',
 'Veto': 0,
 'Class': 0.0,
 'ComC': 0.0,
 'ComR': 0.0,
 'CumHServ': 2190.0,
 'CumSServ': 0.0,
 'Delegate': 0,
 'District': 24.0,
 'DW1': '0,713',
 'FrstConH': 109.0,
 'FrstConS': 0.0,
 'Gender': 0,
 'MemberID': '20532-112-200',
 'MRef': 0,
 'NameFirst': 'Kenny',
 'NameFull': 'Kenny Ewell Marchant',
 'NameLast': 'Marchant',
 'Party': 200.0,
 'PooleID': 20532.0,
 'Postal': 'TX',
 'State': 49,
 'URL': '',
 'ChRef': 0,
 'RankRef': 0,
 'PassH': 0,
 'PassS': 0,
 'PLawDate': 0,
 'PLawNum': 0,
 'ImpBill': nan,
 'SubChRef': 0.0,
 'SubRankRef': 0.0,
 'Majority': 1,
 'Report

In [51]:
for bill in remaining_final:
    if bill['URL'] == '':
        print('No URL for: ', bill['BillID'])
    else:
        bill['Summary'] = scraper(bill['URL'])
        
    query = db.insert(current_bills_table)
    connection.execute(query, bill)

https://www.congress.gov/bill/112th-congress/senate-bill/1629


MissingSchema: Invalid URL 'nan': No schema supplied. Perhaps you meant http://nan?

In [24]:
# query = db.insert(bills)

In [25]:
# connection.execute(query, final)

In [26]:
# Stopped at 17778

#use reversed(final) to start from the other side.